In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# --- ¡NUEVO! --- 
# Importamos las capas necesarias para la CNN y los Callbacks
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
import math
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'tensorflow_datasets'

### Fashion-MNIST
Fashion-MNIST es un conjunto de datos de imágenes de artículos de Zalando que consta de un conjunto de entrenamiento de 60 000 ejemplos y un conjunto de prueba de 10 000 ejemplos. Cada ejemplo es una imagen en escala de grises de 28x28, asociada con una etiqueta de 10 clases.

In [ ]:
#Descargar set de datos de Fashion MNIST de Zalando
datos, metadatos = tfds.load('fashion_mnist', as_supervised=True, with_info=True)

In [ ]:
#Obtenemos en variables separadas los datos de entrenamiento (60k) y pruebas (10k)
datos_entrenamiento, datos_pruebas = datos['train'], datos['test']

In [ ]:
#Etiquetas de las 10 categorias posibles
nombres_clases = metadatos.features['label'].names
print(nombres_clases)

In [ ]:
# --- CELDA MODIFICADA --- 

# Funcion de normalizacion (Pasar de 0-255 a 0-1)
def normalizar(imagenes, etiquetas):
  imagenes = tf.cast(imagenes, tf.float32)
  imagenes /= 255 #reducción de [0-255] a [0,1]
  return imagenes, etiquetas

# Normalizar los datos de entrenamiento y pruebas con la funcion que hicimos
datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_pruebas = datos_pruebas.map(normalizar)

# ¡NUEVO! Separar 10k datos de entrenamiento para validación
num_ej_entrenamiento = metadatos.splits["train"].num_examples
num_ej_pruebas = metadatos.splits["test"].num_examples
print(f"Ejemplos de entrenamiento originales: {num_ej_entrenamiento}")
print(f"Ejemplos de prueba: {num_ej_pruebas}")

num_ej_validacion = 10000
num_ej_entrenamiento_nuevo = num_ej_entrenamiento - num_ej_validacion
print(f"Ejemplos de entrenamiento nuevos: {num_ej_entrenamiento_nuevo}")
print(f"Ejemplos de validación: {num_ej_validacion}")

datos_validacion = datos_entrenamiento.take(num_ej_validacion)
datos_entrenamiento = datos_entrenamiento.skip(num_ej_validacion)

#Agregar a cache (usar memoria en lugar de disco, entrenamiento mas rapido)
datos_entrenamiento = datos_entrenamiento.cache()
datos_validacion = datos_validacion.cache()
datos_pruebas = datos_pruebas.cache()

In [ ]:
#Mostrar una imagen de los datos de pruebas (ahora del set de 50k)
for imagen, etiqueta in datos_entrenamiento.take(1):
  break
imagen = imagen.numpy().reshape((28,28)) #Redimensionar

#Dibujar dibujar
plt.figure()
plt.imshow(imagen, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
#Dibujar mas
plt.figure(figsize=(10,10))
for i, (imagen, etiqueta) in enumerate(datos_entrenamiento.take(25)):
  imagen = imagen.numpy().reshape((28,28))
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(imagen, cmap=plt.cm.binary)
  plt.xlabel(nombres_clases[etiqueta])
plt.show()

### Crear el modelo (CNN)

In [ ]:
# --- ¡NUEVA CELDA DE MODELO! ---
# Esta es una Red Neuronal Convolucional (CNN)

modelo = tf.keras.Sequential([
    
    # 1. Capa Convolucional: Busca 32 patrones (filtros) de 3x3.
    # La entrada sigue siendo nuestra imagen de 28x28x1.
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    
    # 2. Capa de Pooling: Reduce el tamaño de la imagen a la mitad (14x14).
    MaxPooling2D((2, 2)),

    # 3. Segunda Capa Convolucional: Busca 64 patrones.
    Conv2D(64, (3, 3), activation='relu'),
    
    # 4. Segunda Capa de Pooling: Reduce de nuevo (7x7).
    MaxPooling2D((2, 2)),

    # 5. Aplanar (Flatten): Aplana las características aprendidas (7x7x64).
    Flatten(),
    
    # 6. Capa Densa: 128 neuronas para clasificar los patrones.
    Dense(128, activation=tf.nn.relu),
    
    # (Opcional) Dropout: Ayuda a prevenir el sobreajuste.
    Dropout(0.2),

    # 7. Capa de Salida: 10 clases (sin cambios).
    Dense(10, activation=tf.nn.softmax)
])

# Compilar el modelo
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Imprime un resumen de tu nueva arquitectura
modelo.summary()

In [ ]:
# --- CELDA MODIFICADA --- 
# Preparamos los lotes para los 3 sets de datos

TAMANO_LOTE = 32

# Shuffle y repeat hacen que los datos esten mezclados de manera aleatoria
datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_ej_entrenamiento_nuevo).batch(TAMANO_LOTE)

# Preparamos los lotes de validación y pruebas
datos_validacion = datos_validacion.batch(TAMANO_LOTE)
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)

### Entrenamiento del modelo (Con Validación y EarlyStopping)

In [ ]:
# --- ¡NUEVA CELDA DE ENTRENAMIENTO! ---

# Este callback monitorea la precisión de validación ('val_accuracy')
# Si no mejora durante 3 épocas seguidas ('patience=3'), detiene el entrenamiento.
# 'restore_best_weights=True' se asegura de que el modelo final sea el de la mejor época.
early_stop = EarlyStopping(
    monitor='val_accuracy', 
    patience=3, 
    restore_best_weights=True
)

# Entrenar
historial = modelo.fit(
    datos_entrenamiento, 
    epochs=20, # Ponemos un número alto, EarlyStopping decidirá cuándo parar
    steps_per_epoch= math.ceil(num_ej_entrenamiento_nuevo / TAMANO_LOTE),
    validation_data=datos_validacion, # ¡Usamos el set de validación!
    validation_steps=math.ceil(num_ej_validacion / TAMANO_LOTE),
    callbacks=[early_stop] # ¡Añadimos el callback!
)

# Evalúa el modelo final con los datos de prueba para ver la precisión final
print("\n--- Evaluación Final con Datos de Prueba ---")
test_loss, test_accuracy = modelo.evaluate(datos_pruebas, steps=math.ceil(num_ej_pruebas/TAMANO_LOTE))
print(f'\nPrecisión final en pruebas: {test_accuracy * 100:.2f}%')

### Predicción
Predicción utilizando nuestro modelo de red neuronal convolucional

In [ ]:
#Pintar una cuadricula con varias predicciones, y marcar si fue correcta (azul) o incorrecta (roja)

for imagenes_prueba, etiquetas_prueba in datos_pruebas.take(1):
  imagenes_prueba = imagenes_prueba.numpy()
  etiquetas_prueba = etiquetas_prueba.numpy()
  predicciones = modelo.predict(imagenes_prueba)

def graficar_imagen(i, arr_predicciones, etiquetas_reales, imagenes):
  arr_predicciones, etiqueta_real, img = arr_predicciones[i], etiquetas_reales[i], imagenes[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img[...,0], cmap=plt.cm.binary)

  etiqueta_prediccion = np.argmax(arr_predicciones)
  if etiqueta_prediccion == etiqueta_real:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(nombres_clases[etiqueta_prediccion],
                                100*np.max(arr_predicciones),
                                nombres_clases[etiqueta_real]),
                                color=color)

def graficar_valor_arreglo(i, arr_predicciones, etiqueta_real):
  arr_predicciones, etiqueta_real = arr_predicciones[i], etiqueta_real[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  grafica = plt.bar(range(10), arr_predicciones, color="#777777")
  plt.ylim([0, 1])
  etiqueta_prediccion = np.argmax(arr_predicciones)

  grafica[etiqueta_prediccion].set_color('red')
  grafica[etiqueta_real].set_color('blue')

filas = 5
columnas = 5
num_imagenes = filas*columnas
plt.figure(figsize=(2*2*columnas, 2*filas))
for i in range(num_imagenes):
  plt.subplot(filas, 2*columnas, 2*i+1)
  graficar_imagen(i, predicciones, etiquetas_prueba, imagenes_prueba)
  plt.subplot(filas, 2*columnas, 2*i+2)
  graficar_valor_arreglo(i, predicciones, etiquetas_prueba)